In [32]:
# 1. 读取文件
# 2. 词元化 将文文本数据拆分成字符或者单词
# 3. 建立词表  词表包含单词或者字符的频率， 从id到单词或者字符的索引 数组表示，从单词或者字符的索引到单词 字典表示
#              文章的长度__len__  __getitem__ 返回token所在的索引  根据索引返回对应的单词或者字符 用<unk>表示空

import re 

def readfile():
    lines=[]
    with open("../data/timemachine.txt") as f:
        for line in f:
            lines.append(re.sub("[^A-Za-z]+",' ',line).strip().lower())
    return lines

lines=readfile()
for i in range(2):
    print(lines[i])


the time machine by h g wells



In [33]:
def tokenize(lines,type='word'):
    if type=='word':
        return [ line.split() for line in lines]
    elif type=='char':
        return [list(line) for line in lines]
    else:
        return None

tokens=tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [34]:
import collections

class Vocab():
    def __init__(self,tokens):
        words=[word for token in tokens for word in token]
        counter=collections.Counter(words)
        self._token_freq=sorted(counter.items(),key=lambda x: x[1],reverse=True)
        
        self.idx_to_token=['<unk>']
        self.token_to_idx={0:'<unk>'}
        for token,freq in self._token_freq:
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token]=len(self.idx_to_token)-1
    
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self,tokens):
        return [self.token_to_idx.get(token,self.unk) for token in tokens]

    def to_tokens(self,indices):
        return [ self.idx_to_token[index] for index in indices]

    @property
    def unk(self):
        return 0

    @property
    def token_freq(self):
        return self._token_freq
    
vocab=Vocab(tokens)
print(vocab.token_freq[:10])
print(list(vocab.token_to_idx.items())[:10])
for i in range(10):
    print("文本：",tokens[i])
    print("索引：",vocab[tokens[i]])


[('the', 2261), ('i', 1267), ('and', 1245), ('of', 1155), ('a', 816), ('to', 695), ('was', 552), ('in', 541), ('that', 443), ('my', 440)]
[(0, '<unk>'), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]
文本： ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引： [1, 19, 50, 40, 2183, 2184, 400]
文本： []
索引： []
文本： []
索引： []
文本： []
索引： []
文本： []
索引： []
文本： ['i']
索引： [2]
文本： []
索引： []
文本： []
索引： []
文本： ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
索引： [1, 19, 71, 16, 37, 11, 115, 42, 680, 6, 586, 4, 108]
文本： ['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
索引： [7, 1420, 5, 2185, 587, 6, 126, 25, 330, 127, 439, 3]


In [36]:
def load_corpus_time_machine():
    lines=readfile()
    tokens=tokenize(lines,'char')
    vocab=Vocab(tokens)

    corpus=[ch for token in tokens for ch in token]
    return corpus,vocab

corpus,vocab=load_corpus_time_machine()
len(corpus),len(vocab)

(170580, 28)